In [ ]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split

def preprocess_audio(audio_file_path, max_length=500):
    try:

        audio, _ = librosa.load(audio_file_path, sr=16000)
        mfccs = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=40)
        if mfccs.shape[1] < max_length:
            mfccs = np.pad(mfccs, ((0, 0), (0, max_length - mfccs.shape[1])), mode='constant')
        else:
            mfccs = mfccs[:, :max_length]
        mfccs_normalized = (mfccs - np.mean(mfccs)) / np.std(mfccs)
        return mfccs_normalized
    except Exception as e:
        print(f"Error encountered while processing file: {audio_file_path}")
        return None


def load_data(directory):
    data = []
    labels = []
    for label, folder in enumerate(['Real', 'Fake']):
        folder_path = os.path.join(directory, folder)
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            features = preprocess_audio(file_path)
            if features is not None:
                data.append(features)
                labels.append(label)
    return np.array(data), np.array(labels)


dataset_dir = "/content/drive/MyDrive/deepfake/Training"


train_dir = os.path.join(dataset_dir)
x_train, y_train = load_data(train_dir)

#val_dir = os.path.join(dataset_dir, "validation")
#x_val, y_val = load_data(val_dir)

# Check the shapes of the loaded data
print("Training data shape:", x_train.shape)
print("Training labels shape:", y_train.shape)
#print("Validation data shape:", x_val.shape)
#print("Validation labels shape:", y_val.shape)


Training data shape: (1000, 40, 500)
Training labels shape: (1000,)


In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten

# Define the model
model = Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(40, 500)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(units=64, return_sequences=True),
    LSTM(units=64),
    Dense(units=128, activation='relu'),
    Dropout(rate=0.5),
    Dense(units=1, activation='sigmoid')
])



In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:

# Train the model
history = model.fit(x_train, y_train, epochs=10, batch_size=32)


Epoch 1/10
32/32 [==============================] - 7s 29ms/step - loss: 0.5251 - accuracy: 0.7480
Epoch 2/10
32/32 [==============================] - 1s 27ms/step - loss: 0.2827 - accuracy: 0.9050
Epoch 3/10
32/32 [==============================] - 1s 27ms/step - loss: 0.2500 - accuracy: 0.9180
Epoch 4/10
32/32 [==============================] - 1s 26ms/step - loss: 0.1968 - accuracy: 0.9290
Epoch 5/10
32/32 [==============================] - 1s 29ms/step - loss: 0.1523 - accuracy: 0.9470
Epoch 6/10
32/32 [==============================] - 1s 27ms/step - loss: 0.1538 - accuracy: 0.9500
Epoch 7/10
32/32 [==============================] - 1s 42ms/step - loss: 0.1266 - accuracy: 0.9570
Epoch 8/10
32/32 [==============================] - 1s 25ms/step - loss: 0.0861 - accuracy: 0.9690
Epoch 9/10
32/32 [==============================] - 1s 27ms/step - loss: 0.0574 - accuracy: 0.9810
Epoch 10/10
32/32 [==============================] - 1s 29ms/step - loss: 0.0542 - accuracy: 0.9820


In [ ]:

# Evaluate the model
loss, accuracy = model.evaluate(x_train, y_train)
print("Training Loss:", loss)
print("Training Accuracy:", accuracy)


32/32 [==============================] - 1s 10ms/step - loss: 0.0353 - accuracy: 0.9890
Training Loss: 0.035285353660583496
Training Accuracy: 0.9890000224113464


In [ ]:
import librosa
import numpy as np

def preprocess_audio(audio_file_path, max_length=500):
    try:
        # Load audio file
        audio, _ = librosa.load(audio_file_path, sr=16000)
        # Extract MFCC features
        mfccs = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=40)
        # Pad or trim MFCCs to ensure a consistent shape
        if mfccs.shape[1] < max_length:
            mfccs = np.pad(mfccs, ((0, 0), (0, max_length - mfccs.shape[1])), mode='constant')
        else:
            mfccs = mfccs[:, :max_length]
        # Normalize MFCC features
        mfccs_normalized = (mfccs - np.mean(mfccs)) / np.std(mfccs)
        return mfccs_normalized
    except Exception as e:
        print(f"Error encountered while processing file: {audio_file_path}")
        return None

def predict(audio_file_path, model, threshold=0.5):
    # Preprocess the audio file
    processed_audio = preprocess_audio(audio_file_path)
    if processed_audio is not None:
        # Reshape the input for the model
        input_data = np.expand_dims(processed_audio, axis=0)
        # Make predictions
        prediction = model.predict(input_data)
        # Determine class based on threshold
        if prediction >= threshold:
            return "Fake"
        else:
            return "Real"
    else:
        return None

# Example usage
audio_file_path = "/content/drive/MyDrive/deepfake/Testing/Fake/musk-to-obama.wav"
prediction = predict(audio_file_path, model)
print("Prediction:", prediction)



1/1 [==============================] - 0s 40ms/step
Prediction: Real


In [ ]:
# Save the model
model.save("model.h5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
